In [1]:
import os
os.chdir('/Users/lantrieu1964/Documents/GitHub/qbio_490_LanTrieu/analysis_data')

In [2]:
#Import CPTAC & Examine Data Sets Available
import cptac
cptac.list_datasets()

cptac warning: Your version of cptac (1.1.2) is out-of-date. Latest is 1.5.7. Please run 'pip install --upgrade cptac' to update it. (/Users/lantrieu1964/miniconda3/envs/qbio_490_LanTrieu/lib/python3.9/threading.py, line 917)


,Description,Data reuse status,Publication link
Dataset name,,,
Brca,breast cancer,no restrictions,https://pubmed.ncbi.nlm.nih.gov/33212010/
Ccrcc,clear cell renal cell carcinoma (kidney),no restrictions,https://pubmed.ncbi.nlm.nih.gov/31675502/
Colon,colorectal cancer,no restrictions,https://pubmed.ncbi.nlm.nih.gov/31031003/
Endometrial,endometrial carcinoma (uterine),no restrictions,https://pubmed.ncbi.nlm.nih.gov/32059776/
Gbm,glioblastoma,no restrictions,https://pubmed.ncbi.nlm.nih.gov/33577785/
Hnscc,head and neck squamous cell carcinoma,no restrictions,https://pubmed.ncbi.nlm.nih.gov/33417831/
Lscc,lung squamous cell carcinoma,no restrictions,https://pubmed.ncbi.nlm.nih.gov/34358469/
Luad,lung adenocarcinoma,no restrictions,https://pubmed.ncbi.nlm.nih.gov/32649874/
Ovarian,high grade serous ovarian cancer,no restrictions,https://pubmed.ncbi.nlm.nih.gov/27372738/


In [4]:
#Load & Download CCRCC Dataset from CPTAC
cptac.download(dataset = "Ccrcc")
ccrcc = cptac.Ccrcc()
ccrcc.list_data()

Below are the dataframes contained in this dataset and their dimensions:

clinical
	194 rows
	171 columns
CNV
	110 rows
	19285 columns
followup
	352 rows
	27 columns
medical_history
	370 rows
	4 columns
methylation
	107 rows
	15885 columns
phosphoproteomics
	194 rows
	81550 columns
phosphoproteomics_gene
	194 rows
	6127 columns
proteomics
	194 rows
	11710 columns
somatic_mutation
	8350 rows
	3 columns
transcriptomics
	185 rows
	19275 columns


In [8]:
import numpy as np
import pandas as pd

clinical_data = ccrcc.get_clinical()

#Create Mask for RNA Data
rna_data = ccrcc.get_transcriptomics()
rna_mask = rna_data.index.isin(clinical_data.index)
rna_data = rna_data[rna_mask]
rna_data = np.log2(rna_data)
rna_data = rna_data.dropna(axis = 1)
rna_data = rna_data.T.loc[~rna_data.T.index.duplicated(), :].T

#Create Mask for Protein Data
protein_data = ccrcc.get_proteomics()
protein_data.columns = protein_data.columns.get_level_values(0) 
protein_mask = protein_data.index.isin(clinical_data.index)
protein_data = protein_data[protein_mask]
protein_data = protein_data.dropna(axis = 1)
protein_data = protein_data.T.loc[~protein_data.T.index.duplicated(), :].T

#Create Masks for Stage I-III Tumors
stage_mask = np.where(clinical_data.tumor_stage_pathological != "Stage IV", True, False)
clinical_data = clinical_data[stage_mask]
clinical_data = clinical_data[clinical_data.tumor_stage_pathological.notna()]
clinical_data.tumor_stage_pathological.unique()
clinical_data

/Users/lantrieu1964/miniconda3/envs/qbio_490_LanTrieu/lib/python3.9/site-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: divide by zero encountered in log2
  result = func(self.values, **kwargs)


Name,Sample_Tumor_Normal,tumor/normal,gender,age,height_in_cm,height_in_inch,weight_in_kg,weight_in_lb,BMI,race,...,histologic_type_of_normal_tissue,slide_is_free_of_tumor,consistent_with_local_pathology_report,findings_not_consistent_with_local_pathology_report,weight_in_mg,minutes_clamp_1_to_collection,minutes_clamp_2_to_collection,minutes_collection_to_frozen,consistent_with_diagnostic_report,patient_medications
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00004,Tumor,TN,Male,72,170.0,67.0,66.0,145.0,22.80,White,...,NaN,NaN,Yes,NaN,484.0,NaN,NaN,21,Yes,"rivaroxaban,tylenol,Aspirin,glycolax,colace,zocor"
C3L-00010,Tumor,TN,Male,30,177.0,70.0,107.0,236.0,34.15,White,...,NaN,NaN,Yes,NaN,575.0,NaN,NaN,17,Yes,"Rivaroxaban,Esomeprazole ,Tramadol"
C3L-00026,Tumor,TN,Female,65,163.0,64.0,75.0,165.0,28.23,White,...,NaN,NaN,Yes,NaN,212.0,13.0,NaN,13,Yes,"Levothyroxine Sodium,Cyproheptadine HCL,Citrac..."
C3L-00079,Tumor,TN,Male,49,175.0,69.0,116.0,256.0,37.88,White,...,NaN,NaN,Yes,NaN,675.0,NaN,NaN,28,Yes,"ibuprofen,Norco,miralax"
C3L-00088,Tumor,TN,Male,72,178.0,70.0,104.0,229.0,32.82,White,...,NaN,NaN,Yes,NaN,306.0,NaN,NaN,29,Yes,"Lisinopril,Victoza,Tamsulosin,Simvastatin,Quet..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-01646,Tumor,TN,Male,69,170.0,67.0,75.0,165.0,25.00,NaN,...,NaN,NaN,Yes,NaN,303.0,NaN,NaN,20,Yes,"Rosuvastatinum,Humulin N,fenofibratum,torasemi..."
C3N-01648,Tumor,TN,Male,69,176.0,69.0,87.0,192.0,28.00,NaN,...,NaN,NaN,Yes,NaN,129.0,NaN,NaN,8,Yes,"Ramipril,Bisoprolol"
C3N-01649,Tumor,TN,Male,51,173.0,68.0,105.0,231.0,35.00,NaN,...,NaN,NaN,Yes,NaN,228.0,NaN,NaN,29,Yes,"metoprolol,torasemidum,Methyldopum,nitrendypin..."


In [21]:
#Create a New Data Frame that Includes the 5 Most Differentially Expressed RNA Genes & Proteins
top_5_rna_genes = np.abs(rna_data.mean()).sort_values(ascending = False).head(5)
top_5_protein_genes = np.abs(protein_data.mean()).sort_values(ascending = False).head(5)
print("Top 5 RNA Genes:")
print(top_5_rna_genes)
print("Top 5 Protein Genes:")
print(top_5_protein_genes)

combined_gene_data = pd.concat([rna_data[top_5_rna_genes.index], protein_data[top_5_protein_genes.index]], axis = 1)
print("Combined Gene Data:")
print(combined_gene_data)

Top 5 RNA Genes:
Name
ELF5      inf
RAX2      inf
FBXW12    inf
FBXW10    inf
FBXO47    inf
dtype: float64
Top 5 Protein Genes:
Name
FABP7      1.775598
AKR1B10    1.454579
MT1H       1.372425
HMGCS2     1.246962
MT1G       1.230909
dtype: float64
Combined Gene Data:
Name                 ELF5          RAX2        FBXW12        FBXW10  \
Patient_ID                                                            
C3L-00004   -7.135883e+00 -5.248972e+00 -5.720846e+00 -4.792302e+00   
C3L-00010    1.000000e+10 -6.688864e+00  1.000000e+10 -5.647232e+00   
C3L-00011    1.000000e+10  1.000000e+10 -5.040122e+00 -6.111578e+00   
C3L-00026   -2.904599e+00  1.000000e+10 -4.428160e+00 -4.277224e+00   
C3L-00079    1.000000e+10  1.000000e+10 -5.853282e+00  1.000000e+10   
...                   ...           ...           ...           ...   
C3N-00317.N  1.000000e+10  1.000000e+10  1.000000e+10  1.000000e+10   
C3N-00320.N  1.000000e+10  1.000000e+10  1.000000e+10  1.000000e+10   
C3N-00437.N  1.000000e

In [22]:
#Scaling, UMAP Embedding, & Ordinal Encoding
from sklearn.preprocessing import StandardScaler
from umap import UMAP
scaler = StandardScaler()
embedding_unscaled = UMAP().fit_transform(combined_gene_data)
scaled_data = scaler.fit_transform(combined_gene_data)
embedding_scaled = UMAP().fit_transform(scaled_data)

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
unencoded_columns = combined_gene_data
encoded_columns = encoder.fit_transform(combined_gene_data)

In [ ]:
#Compare the 4 Classification Models
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

classifier = KNeighborsClassifier() 
X_train, X_test, y_train, y_test = train_test_split(scaled_data, combined_gene_data, train_size = 0.7)
classifier.fit(X_train, y_train.ravel())
y_pred = classifier.predict(X_test)

regressors_names = [
    'KNeighborsClassifier', 
    'DecisionTreeClassifier', 
    'MLPClassifier', 
    'GaussianNB'
]

regressors = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    MLPClassifier(),
    GaussianNB()
]

regressors_perf = {
    0: [],
    1: [],
    2: [],
    3: []
}

for n in range(10):
    X_train, X_test, y_train, y_test = train_test_split(scaled_data, combined_gene_data, train_size=0.7)
    for i in range(len(regressors)):
        model = regressors[i]
        model.fit(X_train, y_train.ravel())
        y_pred = model.predict(X_test)

        difference = y_pred - y_test
        regressors_perf[i].append(abs(np.mean(difference)))

print('\nAfter 10 simulations, the average error for each regressor is as follows:')
for i in regressors_perf:
    print(f'\t{regressors_names[i]} : {np.mean(regressors_perf[i])}')
    
#DecisionTreeClassifier appears to be the best model.